# MNIST 손글씨 - CNN 2

###  0.9768999814987183
- 2 Conv2D layers
- 1 MaxPooling2D layer
- 1 Fully Connected Network layer
- 1 Output layer

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
seed = 2021
np.random.seed(seed)
tf.random.set_seed(seed)

## 1. 데이터 전처리

In [2]:
from tensorflow.keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train.shape, y_train.shape, X_test.shape, y_test.shape 

11493376/11490434 [==============================] - 0s 0us/step


((60000, 28, 28), (60000,), (10000, 28, 28), (10000,))

In [3]:
# 0과 1 사이 값으로 정규화
X_train = X_train / 255.0
X_test = X_test /255.0

In [4]:
# 0과 1 사이 값으로 정규화
X_train = X_train / 255.0
X_test = X_test /255.0

In [5]:
# 3차원의 데이터를 4차원으로( 데이터는 2차원을 3차원으로)
# X가 28*28이므로 1차원으로 바꾸어 주어야한다. 28*28=784 
X_train = X_train.reshape(-1,28,28, 1)
X_test = X_test.reshape(-1,28, 28, 1)
X_train.shape, X_test.shape

((60000, 28, 28, 1), (10000, 28, 28, 1))

In [6]:
# One hot encoding
from tensorflow.keras.utils import to_categorical
Y_train = to_categorical(y_train)
Y_test = to_categorical(y_test)
Y_train[0]

array([0., 0., 0., 0., 0., 1., 0., 0., 0., 0.], dtype=float32)

## 2. 모델 정의, 설정, 실행, 평가

In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPooling2D # 모델 정의시
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping  # 모델 학습시, collback

In [10]:
model = Sequential([
    Conv2D(32,(3,3), input_shape = (28,28,1), activation='relu'),
    Conv2D(64,(3,3), activation='relu'),
    MaxPooling2D(pool_size =2),
    Flatten(),                           # 2차원을 1차원으로 변환
    Dense(512, activation='relu'),
    Dense(10, activation='softmax') 
     
])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 12, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 9216)              0         
_________________________________________________________________
dense (Dense)                (None, 512)               4719104   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 4,743,050
Trainable params: 4,743,050
Non-trainable params: 0
______________________________________________

In [11]:
model.compile(
    optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy']
)

In [12]:
# 모델 최적화를 위한 설정
modelpath = 'model/best_mnist_cnn_2.h5'
checkpoint = ModelCheckpoint(modelpath, verbose=1, save_best_only=True)  #callbacks 설정
early_stopping = EarlyStopping(patience=10)#callbacks 설정

In [13]:
# 모델 학습
history = model.fit(
    X_train, Y_train, validation_split=0.2,
    epochs=50, batch_size=200, verbose=0,
    callbacks=[checkpoint, early_stopping]
)


Epoch 00001: val_loss improved from inf to 0.32408, saving model to model/best_mnist_cnn1.h5

Epoch 00002: val_loss improved from 0.32408 to 0.23348, saving model to model/best_mnist_cnn1.h5

Epoch 00003: val_loss improved from 0.23348 to 0.19077, saving model to model/best_mnist_cnn1.h5

Epoch 00004: val_loss improved from 0.19077 to 0.15285, saving model to model/best_mnist_cnn1.h5

Epoch 00005: val_loss improved from 0.15285 to 0.13670, saving model to model/best_mnist_cnn1.h5

Epoch 00006: val_loss improved from 0.13670 to 0.12963, saving model to model/best_mnist_cnn1.h5

Epoch 00007: val_loss improved from 0.12963 to 0.11683, saving model to model/best_mnist_cnn1.h5

Epoch 00008: val_loss improved from 0.11683 to 0.11366, saving model to model/best_mnist_cnn1.h5

Epoch 00009: val_loss improved from 0.11366 to 0.10281, saving model to model/best_mnist_cnn1.h5

Epoch 00010: val_loss did not improve from 0.10281

Epoch 00011: val_loss did not improve from 0.10281

Epoch 00012: val_

In [14]:
from tensorflow.keras.models import load_model
best_model = load_model(modelpath)
best_model.evaluate(X_test, Y_test)

313/313 [==============================] - 1s 3ms/step - loss: 0.0849 - accuracy: 0.9769


[0.0848802849650383, 0.9768999814987183]